# Polymarket API Python

**Resources:**
- [Polymarket API Docs](https://docs.polymarket.com/)
- [Gamma API (Market Data)](https://gamma-api.polymarket.com/)
- [Data API (User Data)](https://data-api.polymarket.com/)
- [py-clob-client (3rd Party using Official SDK)](https://github.com/Polymarket/py-clob-client)

## 1. Setup

In [2]:
!pip install py-clob-client requests

  Using cached pycryptodome-3.23.0-cp37-abi3-macosx_10_9_universal2.whl.metadata (3.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.5/587.5 kB 4.6 MB/s  0:00:00
Using cached pycryptodome-3.23.0-cp37-abi3-macosx_10_9_universal2.whl (2.5 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 986.1/986.1 kB 9.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22/22 [py-clob-client]m [cytoolz]dome]


In [3]:
import requests
import time
import json
from pprint import pprint

from py_clob_client.client import ClobClient
from py_clob_client.clob_types import OrderArgs, MarketOrderArgs, OrderType, OpenOrderParams, BalanceAllowanceParams, AssetType
from py_clob_client.order_builder.constants import BUY, SELL

GAMMA_API = "https://gamma-api.polymarket.com"
DATA_API = "https://data-api.polymarket.com"
CLOB_API = "https://clob.polymarket.com"

## 2. Market Discovery

In [4]:
# Fetch active markets sorted by volume
response = requests.get(
    f"{GAMMA_API}/markets",
    params={
        "limit": 10,
        "active": True,
        "closed": False,
        "order": "volume24hr",
        "ascending": False
    }
)
markets = response.json()
print(f"Found {len(markets)} markets\n")

Found 10 markets



In [5]:
for m in markets[:5]:
    print(f"Question: {m['question']}")
    print(f"  Volume 24h: ${m.get('volume24hr', 0):,.0f}")
    print(f"  Liquidity: ${m.get('liquidityNum', 0):,.0f}")
    print(f"  Prices: {m.get('outcomePrices', 'N/A')}")
    print()

Question: Will Trump nominate Arthur Laffer as the next Fed chair?
  Volume 24h: $4,316,186
  Liquidity: $428,959
  Prices: ["0.0005", "0.9995"]

Question: Fed decreases interest rates by 50+ bps after January 2026 meeting?
  Volume 24h: $3,513,741
  Liquidity: $8,708,663
  Prices: ["0.0015", "0.9985"]

Question: Will Trump acquire Greenland before 2027?
  Volume 24h: $2,570,233
  Liquidity: $929,077
  Prices: ["0.125", "0.875"]

Question: Fed decreases interest rates by 25 bps after January 2026 meeting?
  Volume 24h: $2,475,823
  Liquidity: $1,451,263
  Prices: ["0.0135", "0.9865"]

Question: Will Netflix say "Warner Bros" during earnings call?
  Volume 24h: $2,104,755
  Liquidity: $1,795,987
  Prices: ["0.9995", "0.0005"]



## 3. Market Deep Dive

In [7]:
market = markets[1]
market

{'id': '601697',
 'question': 'Fed decreases interest rates by 50+ bps after January 2026 meeting?',
 'conditionId': '0x17815081230e3b9c78b098162c33b1ffa68c4ec29c123d3d14989599e0c2e113',
 'slug': 'fed-decreases-interest-rates-by-50-bps-after-january-2026-meeting',
 'resolutionSource': '',
 'endDate': '2026-01-28T00:00:00Z',
 'liquidity': '8708663.31834',
 'startDate': '2025-09-17T19:03:50.012988Z',
 'image': 'https://polymarket-upload.s3.us-east-2.amazonaws.com/jerome+powell+glasses1.png',
 'icon': 'https://polymarket-upload.s3.us-east-2.amazonaws.com/jerome+powell+glasses1.png',
 'description': 'The FED interest rates are defined in this market by the upper bound of the target federal funds range. The decisions on the target federal fund range are made by the Federal Open Market Committee (FOMC) meetings.\n\nThis market will resolve to the amount of basis points the upper bound of the target federal funds rate is changed by versus the level it was prior to the Federal Reserve\'s Janua

In [8]:
print(f"Market: {market['question']}")
print(f"End Date: {market['endDate']}")
print(f"Condition ID: {market['conditionId']}")

Market: Fed decreases interest rates by 50+ bps after January 2026 meeting?
End Date: 2026-01-28T00:00:00Z
Condition ID: 0x17815081230e3b9c78b098162c33b1ffa68c4ec29c123d3d14989599e0c2e113


In [ ]:
clob_token_ids = market.get('clobTokenIds')
clob_token_ids = json.loads(clob_token_ids)
print(f"Token IDs: {clob_token_ids}")

if len(clob_token_ids) >= 2:
    yes_token_id = clob_token_ids[0]
    no_token_id = clob_token_ids[1]
    print(f"YES token: {yes_token_id}")
    print(f"NO token: {no_token_id}")

## 4. Order Book Analysis

In [ ]:
# Read-only client for order book
client = ClobClient(CLOB_API)

In [ ]:
# Fetch order book for YES token
if yes_token_id:
    book = client.get_order_book(yes_token_id)
    
    sorted_bids = sorted(book.bids, key=lambda x: float(x.price), reverse=True)
    sorted_asks = sorted(book.asks, key=lambda x: float(x.price), reverse=False)
    
    print("=== YES Token Order Book ===")
    print(f"\nTop 5 Asks (sell orders):")
    for ask in sorted_asks[:5]:
        print(f"  Price: {ask.price} | Size: {ask.size}")
        
    print(f"\nTop 5 Bids (buy orders):")
    for bid in sorted_bids[:5]:
        print(f"  Price: {bid.price} | Size: {bid.size}")


In [ ]:
# Get midpoint, prices, and spread
mid = client.get_midpoint(yes_token_id)
buy_price = client.get_price(yes_token_id, side="BUY")
sell_price = client.get_price(yes_token_id, side="SELL")
spread = client.get_spread(yes_token_id)

print(f"Midpoint: {mid['mid']}")
print(f"Best ask (buy at): {buy_price['price']}")
print(f"Best bid (sell at): {sell_price['price']}")
print(f"Spread: {spread['spread']}")